In [1]:
import pandas as pd

df = pd.read_csv('../data/kaggle_ds.csv')
df

,Unnamed: 0,DateTime,buyin,tourn_id,table,HandID,table_size,level,playing,seat,...,pot_turn,pot_river,ante,blinds_level,bet_pre,bet_flop,bet_turn,bet_river,result,balance
0,0,2020-06-07 18:15:09,$0.92+$0.08,2929450288,1,215051507105,3,1,3,1,...,30,30,0,0,0,0,0,0,gave up,0
1,1,2020-06-07 18:15:09,$0.92+$0.08,2929450288,1,215051507105,3,1,3,2,...,30,30,0,10,10,0,0,0,gave up,-10
2,2,2020-06-07 18:15:09,$0.92+$0.08,2929450288,1,215051507105,3,1,3,3,...,30,30,0,20,20,0,0,0,took chips,10
3,3,2020-06-07 18:15:23,$0.92+$0.08,2929450288,1,215051520234,3,1,3,1,...,180,180,0,20,60,60,0,0,took chips,60
4,4,2020-06-07 18:15:23,$0.92+$0.08,2929450288,1,215051520234,3,1,3,2,...,180,180,0,0,0,0,0,0,gave up,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102610,102610,2021-03-30 19:00:02,$0.92+$0.08,3162349514,1,225348760558,3,1,3,2,...,560,560,0,10,500,0,0,0,took chips,60
102611,102611,2021-03-30 19:00:02,$0.92+$0.08,3162349514,1,225348760558,3,1,3,3,...,560,560,0,20,20,0,0,0,gave up,-20
102612,102612,2021-03-30 19:00:25,$0.92+$0.08,3162349514,1,225348768503,3,1,3,1,...,1030,1030,0,20,20,50,390,0,lost,-460
102613,102613,2021-03-30 19:00:25,$0.92+$0.08,3162349514,1,225348768503,3,1,3,2,...,1030,1030,0,0,20,50,490,0,won,470


In [2]:
dfs = []

# Tables we can fill:
- Users
- Room
- Transaction
- Skin
- Inventory

## Transactions
- TransactionID
- SenderID
- ReceiverID
- Amount
- DateTime
- Description

In [3]:
import numpy as np

potID = 'TEXAS_HOLDEM'
df_transactions = df[['UserID', 'balance', 'DateTime', 'HandID']].copy()
df_transactions = df_transactions.drop(df_transactions[df_transactions['balance'] == 0].index)
df_transactions['TransactionID'] = df_transactions.index
df_transactions['SenderID'] = np.where(df_transactions['balance'] < 0, df_transactions['UserID'], potID)
df_transactions['ReceiverID'] = np.where(df_transactions['balance'] > 0, df_transactions['UserID'], potID)
df_transactions['Description'] = None

# print(df_transactions.columns)
df_transactions = df_transactions.drop(columns=['UserID']).rename(columns={
    'balance' : 'Amount'
})
dfs.append((
    df_transactions,
    ['TransactionID'],
    ['TransactionID', 'SenderID', 'ReceiverID', 'Amount', 'DateTime', 'Description'],
    'transaction'
))
df_transactions

,Amount,DateTime,HandID,TransactionID,SenderID,ReceiverID,Description
1,-10,2020-06-07 18:15:09,215051507105,1,676c5599,TEXAS_HOLDEM,None
2,10,2020-06-07 18:15:09,215051507105,2,TEXAS_HOLDEM,7055f691,None
3,60,2020-06-07 18:15:23,215051520234,3,TEXAS_HOLDEM,fa538846,None
5,-60,2020-06-07 18:15:23,215051520234,5,7055f691,TEXAS_HOLDEM,None
6,60,2020-06-07 18:16:01,215051555272,6,TEXAS_HOLDEM,fa538846,None
...,...,...,...,...,...,...,...
102610,60,2021-03-30 19:00:02,225348760558,102610,TEXAS_HOLDEM,c98315f0,None
102611,-20,2021-03-30 19:00:02,225348760558,102611,e6bfd5eb,TEXAS_HOLDEM,None
102612,-460,2021-03-30 19:00:25,225348768503,102612,fa538846,TEXAS_HOLDEM,None
102613,470,2021-03-30 19:00:25,225348768503,102613,TEXAS_HOLDEM,c98315f0,None


## Skin & Room
### Skin
- SkinID
- Image (path)
- Description

### Room
- RoomID
- Log (path)
- ChatLog (path)

In [4]:
STORAGE_DIR = '../data/imgs/'
df_skins = pd.DataFrame({
    'SkinID' : [f's{str(i)}' for i in range(5)],
    'Image' : [f'{STORAGE_DIR}skin{i}.jpg' for i in range(5)],
    'Description' : [
        'secret',
        'default',
        'red',
        'blue',
        'green'
    ]
})
dfs.append((
    df_skins,
    ['SkinID'],
    ['SkinID', 'Image', 'Description'],
    'skin'
))
df_skins

,SkinID,Image,Description
0,s0,../data/imgs/skin0.jpg,secret
1,s1,../data/imgs/skin1.jpg,default
2,s2,../data/imgs/skin2.jpg,red
3,s3,../data/imgs/skin3.jpg,blue
4,s4,../data/imgs/skin4.jpg,green


In [5]:
LOGS_DIR = '../data/tmp/'
num_rooms = 4
df_rooms = pd.DataFrame({
    'RoomID' : [f'r{str(i)}' for i in range(num_rooms)],
    'Log' : [f'{LOGS_DIR}log{i}.txt' for i in range(num_rooms)],
    'ChatLog' : [f'{LOGS_DIR}chatlog{i}.txt' for i in range(num_rooms)]
})
dfs.append((
    df_rooms,
    ['RoomID'],
    ['RoomID', 'Log', 'ChatLog'],
    'room'
))
df_rooms

,RoomID,Log,ChatLog
0,r0,../data/tmp/log0.txt,../data/tmp/chatlog0.txt
1,r1,../data/tmp/log1.txt,../data/tmp/chatlog1.txt
2,r2,../data/tmp/log2.txt,../data/tmp/chatlog2.txt
3,r3,../data/tmp/log3.txt,../data/tmp/chatlog3.txt


## Users
- UserID (add `TEXAS_HOLDEM` id)
- Pass
- Balance
- CurrentSkin
- RoomID

In [6]:

import random, string

house = {'UserID' : [potID], 'Pass' : ["whatareyoudoinghere"], 'Balance' : [-1], 'CurrentSkin' : ["s0"], 'RoomID' : [None]}

df_users = df[['UserID', 'balance']].groupby(['UserID']).sum().reset_index().copy()
skins = df_skins['SkinID'].array
df_users['CurrentSkin'] = [random.choices(skins, [0.005, .695, .1, .1, .1])[0] for i in range(len(df_users))]
df_users['RoomID'] = [f'r{str(i % 4)}' if i < 16 else None for i in range(len(df_users))]
def random_str(size): 
    return ''.join(random.choices(string.ascii_letters + string.digits, k=size))
df_users['Pass'] = [random_str(random.randrange(8, 15)) for i in range(len(df_users))]
df_users = df_users.rename(columns={'balance' : 'Balance'})
df_users = pd.concat([df_users, pd.DataFrame(house)], ignore_index=True)

dfs.append((
    df_users,
    ['UserID'],
    ['UserID', 'Pass','CurrentSkin', 'RoomID', 'Balance'],
    'user'
))
df_users[df_users['CurrentSkin'] == "s1"]

,UserID,Balance,CurrentSkin,RoomID,Pass
0,00021ae5,-500,s1,r0,1iLyi2Y5
1,0006db09,-500,s1,r1,7G1lNerkJOAdq
2,001af494,600,s1,r2,4RA2EdDGC
5,004ec8a3,-500,s1,r1,ZndbZtONocIkX
6,005dfb56,-500,s1,r2,KLzkn0A3lkd
...,...,...,...,...,...
3403,fee3805e,-500,s1,None,1iSy0GNc3u
3405,fefabe4a,-500,s1,None,5AllmXLIc
3406,ff7ca16d,-500,s1,None,LK8VwGmxLf
3408,ffc62332,-500,s1,None,OIrAmftUtV9ri


## Inventory
- match inventory to users current skin
- give everyone default

attributes:
- UserID
- SkinID

In [7]:
df_inventory = df_users[['UserID', 'CurrentSkin']].copy()
df_inventory = df_inventory[df_inventory['UserID'] != potID]
df_inventory = df_inventory.rename(columns={'CurrentSkin' : 'SkinID'})

df_default_inv = pd.DataFrame({
    'UserID' : [u for u in df_inventory['UserID']],
    'SkinID' : ['s1' for _ in df_inventory['UserID']]
})
# add default skin to each, if alr in delete duplicates
df_inventory = pd.concat([df_inventory, df_default_inv]).drop_duplicates()
df_inventory['SkinID'] = df_inventory['SkinID'].astype(str)
df_inventory['UserID'] = df_inventory['UserID'].astype(str)

dfs.append((
    df_inventory,
    ['UserID', 'SkinID'],
    ['UserID', 'SkinID'],
    'inventory'
))
df_inventory


,UserID,SkinID
0,00021ae5,s1
1,0006db09,s1
2,001af494,s1
3,00394ea7,s4
4,004afe29,s4
...,...,...
3396,fe9b0514,s1
3399,feab68ba,s1
3400,feb1e888,s1
3404,feea9f41,s1


In [8]:
from gcp_csv_import_format import format_df_to_csv

for d, pk, o, p in dfs:
    format_df_to_csv(d, pk, o, f'../data/{p}.csv')


c:\Users\ethan\Documents\uiuc\cs411\fa24-cs411-team074-floor\scripts\gcp_csv_import_format.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tmp'] = [0 for _ in range(len(df))]
